# Preliminaries

In [ ]:
# For data frames:
import pandas as pd

# For arrays:
import numpy as np

# For optimization:
import scipy

In [ ]:
from sklearn.metrics import confusion_matrix, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
# For plotting:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# For OLS regression:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor


---

# Supervised learning using scikit-learn

Supervised learning is a type of machine learning (ML) that involves predicting a ground truth. And we must have some examples in order to "supervise" the learning algorithm.

We'll demonstrate this by predicting whether borrowers will default on a loan. We know the ground truth: whether they have or not. And we will supervise the algorithm as it learns how to predict this fact.

This task is a **classification** task: the loan can be in a discrete state of defaulted or not defaulted. If we are trying to predict which state/**class** a loan will end up in, it's a classification problem.

## Load loans dataset

In [ ]:
loans_df = pd.read_csv('Loan_Default.csv')

In [ ]:
len(loans_df)

In [ ]:
loans_df.info()

In [ ]:
loans_df.head()

In [ ]:
loans_df['Default'].hist()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

## ML-specific cleaning & processing

In [ ]:
# A LoanID column will not generalize, and should be excluded
loans_df.drop(columns=['LoanID'], inplace=True)
loans_df.columns.values

In [ ]:
# These 3 columns are Yes/No...
print(loans_df['HasMortgage'].unique())
print(loans_df['HasDependents'].unique())
print(loans_df['HasCoSigner'].unique())


In [ ]:
# ... so convert them to boolean True/False. (or 1/0 would also work)
loans_df['HasMortgage'] = (loans_df['HasMortgage'] == 'Yes')
loans_df['HasDependents'] = (loans_df['HasDependents'] == 'Yes')
loans_df['HasCoSigner'] = (loans_df['HasCoSigner'] == 'Yes')

In [ ]:
loans_df.head()

### One-hot encoding

There are 4 non-binary text columns remaining that need to be converted to a numerical format somehow.

One-hot encoding is a standard way of doing so.

In [ ]:
print(loans_df['Education'].unique())
print(loans_df['EmploymentType'].unique())
print(loans_df['MaritalStatus'].unique())
print(loans_df['LoanPurpose'].unique())

In [ ]:
# Demonstration of one-hot encoding on the Education column
temp_df = pd.get_dummies(data = loans_df, columns=['Education'])

print(temp_df.columns.values)

temp_df.head()


In [ ]:
# We'll run it on all 4 columns:
loans_df = pd.get_dummies(data = loans_df, columns=['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose'])

In [ ]:
# Everything is now a bool, int or float
loans_df.info()

### Splitting columns into Y,X

In [ ]:
# Split data into the labels Y and predictors X

# What we want to predict
Y = loans_df['Default']

# Everything else
X = loans_df.drop(columns=['Default'], axis=1)


In [ ]:
print(Y.shape)
print(X.shape)

### Splitting rows into training & test samples

In [ ]:
# Split the data into training and test/holdout/out-of-sample sets
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, random_state=42)

In [ ]:
print(train_Y.shape)
print(train_X.shape)

In [ ]:
print(test_Y.shape)
print(test_X.shape)

## Training and evaluating a RandomForestClassifier

### Training on the training data

In [ ]:
# Create a classifier using a RandomForest algorithm...
model = RandomForestClassifier(random_state=42, n_estimators=10)

# ... and fit it to our training data
model.fit(train_X, train_Y)

### Evaluating on the same training data

In [ ]:
# Use this trained model to generate predictions for the training data
train_Y_pred = model.predict(train_X)
train_Y_pred

In [ ]:
# How well do those predictions perform?
confusion = confusion_matrix(y_true = train_Y, y_pred = train_Y_pred)
confusion

In [ ]:
true_negatives, false_positives, false_negatives, true_positives = confusion.ravel()

print('Count of true negatives (predicted 0, true 0)  =  ' + str(true_negatives))
print('Count of false negatives (predicted 0, true 1)  =  ' + str(false_negatives))
print('Count of false positives (predicted 1, true 0)  =  ' + str(false_positives))
print('Count of true positives (predicted 1, true 1)  =  ' + str(true_positives))

In [ ]:
# Simpler measure of classification accuracy:
(train_Y == train_Y_pred).mean()

### Evaluating out-of-sample on the test/holdout data

In [ ]:
# Use this trained model to generate predictions for the test data
test_Y_pred = model.predict(test_X)
test_Y_pred

In [ ]:
# How well do those predictions perform?
confusion = confusion_matrix(y_true = test_Y, y_pred = test_Y_pred)
confusion

In [ ]:
true_negatives, false_positives, false_negatives, true_positives = confusion.ravel()

print('Count of true negatives (predicted 0, true 0)  =  ' + str(true_negatives))
print('Count of false negatives (predicted 0, true 1)  =  ' + str(false_negatives))
print('Count of false positives (predicted 1, true 0)  =  ' + str(false_positives))
print('Count of true positives (predicted 1, true 1)  =  ' + str(true_positives))

In [ ]:
# Simpler measure of classification accuracy:
print('Prediction accuracy rate = ', str((test_Y == test_Y_pred).mean()))

In [ ]:
RocCurveDisplay.from_estimator(model, test_X, test_Y)

### Let's try with a different set of hyperparameters

In [ ]:
# Create & fit
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(train_X, train_Y)

# Evaluate on the test data
test_Y_pred = model.predict(test_X)
print('Prediction accuracy rate = ', str((test_Y == test_Y_pred).mean()))

RocCurveDisplay.from_estimator(model, test_X, test_Y)

Using `n_estimators=100` produces better out-of-sample results than using `n_estimators=10`.

There are other hyperparameters that could be set, too. See the full list at 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


How about systematically picking the value of these hyperparameters? This is called "hyperparameter tuning", and we can do this with cross-validation. 

It can be computationally expensive, so I will leave cross-validation for a homework exercise later.


### Explaining the most important features

Determining the most important "features" (i.e. predictive variables) and how the model arrives at its predictions is an important ongoing area of research in ML. There are both model-specific and general-purpose ways of getting at both questions.

The RandomForest classification algorithm has a convenient way of describing these:




In [ ]:
importances_df = pd.DataFrame({
    'feature_name' : train_X.columns.values
    , 'feature_importance' : model.feature_importances_
})
importances_df.sort_values(by='feature_importance', ascending=False)

A few observations:
* As you can see, we don't learn much about the direction of the effect (although we can guess, in the case of credit scores). 
* This is dependent on the RandomForest algorithm; the code above won't work for other algorithms.

There is no silver bullet. In my own research, I've found [SHAP values](https://shap.readthedocs.io/en/latest/index.html) are very helpful, but it depends on the application, so you'll just have to experiment.

## Let's try another algorithm: a Linear Support Vector Machine classifier 

In [ ]:
# Create & fit
model = LinearSVC(random_state=42, penalty='l2')
model.fit(train_X, train_Y)

# Evaluate on the test data
test_Y_pred = model.predict(test_X)
print('Prediction accuracy rate = ', str((test_Y == test_Y_pred).mean()))

RocCurveDisplay.from_estimator(model, test_X, test_Y)

## Let's try scaling the data first

Scikit-learn allows us to compose a preprocessing step (and more) followed by an ML model.

Here, the preprocessing step computes the z-scores of the data (zero mean, unit variance), which often improves the performance of ML models.

In [ ]:
# "model" is now a "pipeline"
model = make_pipeline(
    StandardScaler(),
    LinearSVC(random_state=42, penalty='l2')
)
model.fit(train_X, train_Y)

# Evaluate on the test data
test_Y_pred = model.predict(test_X)
print('Prediction accuracy rate = ', str((test_Y == test_Y_pred).mean()))

RocCurveDisplay.from_estimator(model, test_X, test_Y)

## Cross-validation

Here's a schematic illustration of how cross-validation splits the data to help find a good set of hyperparameters:

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" />

And here's some code to run it easily using scikit-learn.

(Be careful if you are working with timeseries data, or data with a time dimension: you should take that into account, as in this example: https://otexts.com/fpp3/tscv.html)


In [ ]:
# 5-fold cross validation for a RandomForestClassifier
if False:

    # Step 1: run cross-validation on the training data
    # This involves splitting the data 5 times and fitting 4*3=12 models on each split ("fold"), 
    # for a total of 5*12 = 60 fits. This will take some time!
    # Then it will pick the model-hyperparameter combination that performs best on average across the 5 splits, and refit to the entire training data.
    # This will take some time!
    number_of_cross_validation_splits = 5
    possible_hyperparameter_values = {
        'n_estimators': [20, 50, 100, 200]
        , 'max_depth': [5, 7, 9]
    }
    cv_model = GridSearchCV(RandomForestClassifier, possible_hyperparameter_values, cv=number_of_cross_validation_splits)
    cv_model.fit(train_X, train_Y)
    
    # Step 2: print out the optimal parameter values
    print(cv_model.best_params_) 
    # Similarly you can inspect the grid scores using cv_model.cv_results_
    
    # Step 3: Evaluate optimal model on the test data
    test_Y_pred = cv_model.predict(test_X)
    print('Prediction accuracy rate = ', str((test_Y == test_Y_pred).mean()))
    
    RocCurveDisplay.from_estimator(cv_model, test_X, test_Y)


---

# OLS Regression using statsmodel

In [ ]:
howell_df = pd.read_csv('Howell1.csv', delimiter=';')

This dataset consists of measurements of the Khosan people:
* Sex
* Age (years)
* Weight (kg)
* Total height (cm)

In [ ]:
# Start by looking at adults
howell_adults_df = howell_df[howell_df['age'] >= 18] 

In [ ]:
howell_adults_df

In [ ]:
howell_adults_df.describe()

In [ ]:
sns.regplot(data=howell_adults_df, x='weight', y='height')

## Running a univariate regression

In [ ]:
model = smf.ols(formula="height ~ weight", data=howell_adults_df)

In [ ]:
result = model.fit()

In [ ]:
print(result.summary())

## Running a multivariate regression

In [ ]:
model = smf.ols(formula="height ~ weight + age + male", data=howell_df)
result = model.fit()
print(result.summary())

## Worried about heteroscedasticity? 

Heteroscedasticity is a major concern in econometrics: `statsmodels` allows you to use different methods to compute your standard errors to deal with this:

In [ ]:
rob_result = model.fit().get_robustcov_results(cov_type='HC3')

In [ ]:
print(rob_result.summary())

## Worried about outliers?

There aren't any extreme outliers in our data. If there were, and they were distorting the regression results, some fields are fine with excluding them or [winsorizing them](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html) - as long as this is transparently reported in your study, of course. Otherwise, consider using [robust regression](https://www.statsmodels.org/stable/rlm.html) models.

## Beyond adults

In [ ]:
# Sub-sample of adults
sns.regplot(data=howell_adults_df, x='age', y='height')

In [ ]:
# Entire sample
sns.regplot(data=howell_df, x='age', y='height')

In [ ]:
model = smf.ols(formula="height ~ weight + age + male", data=howell_df)
result = model.fit()
print(result.summary())

In [ ]:
# Adding a quadratic term:
model = smf.ols(formula="height ~ weight + age + np.power(age, 2) + male", data=howell_df)
result = model.fit()
print(result.summary())

## Multicollinearity

There is a warning above about potential numerical problems, such as multicollinearity in the variables. 

Let's check for that in a heuristic way using [Variance Inflation Factors](https://en.wikipedia.org/wiki/Variance_inflation_factor):

In [ ]:
variables = model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif 

It seems that the third variable (age) is problematic, so let's run without it:

In [ ]:
# Adding a quadratic term:
model = smf.ols(formula="height ~ weight + np.power(age, 2) + male", data=howell_df)
result = model.fit()
print(result.summary())

In [ ]:
howell_df[['weight', 'age', 'male']].corr()

In [ ]:
variables = model.exog
vif = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif 

## Producing $\LaTeX$ for your research paper

In [ ]:
print(result.summary().as_latex())

---

# Quiz: Revisiting Starbucks 

* Suppose you intend to examine whether sodium contributes to an increase in calories. 
* Also suppose you know sugar and trans fat have been strong factors for high calories. 
* Taken together, you want to see the association between sodium and calories while controlling for sugar and trans fat. 
* (1) Draw three plots, each of which represents the relationship between each of the three variables above and calories. (Use `regplot()` in seaborn)
* (2) Using the `statsmodel` package, print the result of OLS regression whose formula looks like this: `Calories ~ Sodium + Sugars + TransFat`
* (3) Once you control for sugars and tran sfar, is there a significant association between sodium and calories?

In [ ]:
sb_df = pd.read_csv('menu_starbucks.csv')

In [ ]:
# I'll help you with some cleaning of the column names:
sb_df.columns = sb_df.columns.str.replace(r'\([^)]*\)', "").str.rstrip(" ").str.lstrip(" ")

In [ ]:
sb_df.columns

In [ ]:
# TODO: part (1)

In [ ]:
# TODO: part (2)

---

# Arrays: a case study with OLS regression

Let's say we want to regress a variable $y$ against three variables $x_1$, $x_2$, $x_3$, and we have $i=1,..,N$ samples. Then the objective is to estimate three quantities: $\beta_1$, $\beta_2$, $\beta_3$ that work for all the $N$ values of $y$ and $x_1$, $x_2$, $x_3$.

$$y_i = \beta_1 x_{1,i} + \beta_2 x_{2,i} + \beta_3 x_{3,i} \quad\text{for}\quad i=1,..,N$$

Really, we have $N$ rows for which this must hold:
$$y_1 = \beta_1 x_{1,1} + \beta_2 x_{2,1} + \beta_3 x_{3,1}$$
$$\ldots$$
$$y_i = \beta_1 x_{1,i} + \beta_2 x_{2,i} + \beta_3 x_{3,i}$$
$$\ldots$$
$$y_N = \beta_1 x_{1,N} + \beta_2 x_{2,N} + \beta_3 x_{3,N}$$

The system of equations above can be represented using matrix multiplication:

$$ \begin{bmatrix} 
        y_1 \\ \vdots \\ y_i \\ \vdots \\ y_N 
    \end{bmatrix}  = \begin{bmatrix} 
    x_{1,1} & x_{2,1} & x_{3,1} \\
    & \vdots & \\
    x_{1,i} & x_{2,i} & x_{3,i} \\
    & \vdots & \\
    x_{1,N} & x_{2,N} & x_{3,N}
    \end{bmatrix} 
    \begin{bmatrix} 
        \beta_1 \\ \beta_2 \\ \beta_3 
    \end{bmatrix} $$

We'll write these matrices as Y, X, B:
$$ Y = XB $$



Let's solve for $B$. Pre-multiply both sides with $X^T$, the transpose of $X$:

$$ X^T Y = X^T XB $$

Pre-multiply both sides with $(X^TX)^{-1}$, the inverse of the product $X^T X$:

$$ (X^TX)^{-1} X^T Y = (X^TX)^{-1} X^T XB $$

Since $(X^TX)^{-1}$ cancels out $X^T X$, we have our exact solution:

$$ B = (X^TX)^{-1} X^T Y $$

So let's implement multivariate regression ourselves! We just need to know three operations:
* matrix multiplication
* matrix transposition
* matrix inversion

Arrays and the `numpy` package give us what we need.

## Creating matrices from data frames

In [ ]:
X = howell_adults_df[['weight', 'age', 'male']].values

In [ ]:
X.shape

In [ ]:
X

In [ ]:
Y = howell_adults_df[['height']].values

In [ ]:
Y.shape

In [ ]:
Y

## Matrix operations

Here's how to calculate $X^T$, the transpose of X:

In [ ]:
X.T

In [ ]:
X.T.shape

The function `matmul` from package `numpy` (abbreviated `np`) multiplies two matrices. 

So to calculate $X^T X$ we would write:

In [ ]:
np.matmul(X.T, X)


**Quiz** Calculate $X^T Y$

In [ ]:
# TODO: matrix multiplication

The function `linalg.inv` (from the same packages) takes the inverse of a matrix.

So to calculate $(X^T X)^{-1}$ we would write:

In [ ]:
np.linalg.inv(np.matmul(X.T, X))

We're almost there! 

**Quiz** Calculate:

$B = (X^T X)^{-1} X^T Y$

In [ ]:
# TODO: put it all together

You should get coefficient values like the ones below:

In [ ]:
model = smf.ols(formula="height ~ 0 + weight + age + male", data=howell_adults_df)
result = model.fit()
print(result.summary())

**Quiz for later**

Look up (from a textbook) how to add an intercept for the OLS solution, and then modify the above to achieve it.

*Hint:* the formula is exactly the same, you need to make a change to the array `X`.

In [ ]:
# TODO: quiz for later

---

# Optimization: a case study with OLS regression

OLS = Ordinary Least Squares.

"Least squares" because the optimal $\beta$ values are those which minimize the sum of squared errors.

Let's say you pick these values of beta:

In [ ]:
beta_1 = 2
beta_2 = 1
beta_3 = -5

Then you would predict the following values of $y_i$ for each value of $x_1$, $x_2$, $x_3$:

In [ ]:
predictions = beta_1 * howell_adults_df['weight'] + beta_2 * howell_adults_df['age'] + beta_3 * howell_adults_df['male']
predictions

The difference between the true values and the predictions are the errors/residuals:



In [ ]:
errors = howell_adults_df['height'] - predictions
errors

The squared errors are:

In [ ]:
errors**2

So the sum of squared errors is:

In [ ]:
sum(errors**2)

The optimal $\beta$s would minimize this sum of squared errors. This is an optimization problem.

In [ ]:
def sse(betas_list):
    (beta_1, beta_2, beta_3) = betas_list
    predictions = beta_1 * howell_adults_df['weight'] + beta_2 * howell_adults_df['age'] + beta_3 * howell_adults_df['male']
    errors = howell_adults_df['height'] - predictions
    sse = sum(errors**2)
    return sse

In [ ]:
sse([2, 1, -5])

In [ ]:
sse([1, 2, 3])

In [ ]:
scipy.optimize.minimize(sse, [0,0,0])

We should get coefficient values like the ones below:

In [ ]:
model = smf.ols(formula="height ~ 0 + weight + age + male", data=howell_adults_df)
result = model.fit()
print(result.summary())

**Quiz**. 

Change the function `sse` to add an intercept $\beta_0$. 

And then run `scipy.optimize.minimize(sse, [0,0,0,0])`

In [ ]:
# TODO: quiz